In [1]:
from google.colab import files
uploaded = files.upload()

Saving letter-recognition.csv to letter-recognition.csv


In [2]:
# libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
#from functools import partial
#import pylabpip3 install
import numpy as np
import numpy.linalg as npla
# dataset
letters = pd.read_csv("./letter-recognition.csv")

In [3]:
# about the dataset

# dimensions
print("Dimensions: ", letters.shape, "\n")

# data types
print(letters.info())

# head
letters.head()

Dimensions:  (20000, 17) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2br    20000 non-null  int64 
 12  xy2br   20000 non-null  int64 
 13  xege    20000 non-null  int64 
 14  xegvy   20000 non-null  int64 
 15  yege    20000 non-null  int64 
 16  yegvx   20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB
None


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2br,xy2br,xege,xegvy,yege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [4]:
letter_means = letters.groupby('letter').mean()
letter_means.head()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2br,xy2br,xege,xegvy,yege,yegvx
letter,,,,,,,,,,,,,,,,
A,3.337136,6.975919,5.128010,5.178707,2.991128,8.851711,3.631179,2.755387,2.043093,7.802281,2.338403,8.465146,2.771863,6.321926,2.875792,7.468948
B,3.985640,6.962141,5.088773,5.169713,4.596606,7.671018,7.062663,5.366841,5.571802,7.954308,5.506527,6.652742,3.117493,7.919060,6.612272,9.100522
C,4.031250,7.063859,4.701087,5.296196,2.775815,5.437500,7.627717,5.927989,7.177989,8.773098,7.494565,11.947011,1.991848,8.876359,4.080163,8.555707
D,4.023602,7.244720,5.170186,5.288199,4.026087,7.539130,6.806211,5.921739,6.508075,8.166460,5.111801,5.750311,3.365217,7.813665,3.971429,7.628571
E,3.727865,6.944010,4.756510,5.201823,3.679688,5.966146,7.352865,4.223958,7.585938,8.507812,6.242188,10.341146,2.127604,8.298177,6.022135,8.506510


In [5]:
# average feature values
round(letters.drop('letter', axis=1).mean(), 2)
# splitting into X and y

X = letters.drop("letter", axis = 1)
y = letters['letter']
# scaling the features
X_scaled = scale(X)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3)

In [6]:
import random
print(type(X_train))
print((y_train.to_numpy()))

<class 'numpy.ndarray'>
['O' 'Z' 'Z' ... 'X' 'E' 'H']


In [7]:
def markov_samp(X_train, Y_train, k = 5, q = 1.2):

    # Init_sampl = np.concatenate((X_train, np.array()), axis = 0)
    Init_sampl = np.concatenate((np.vstack(X_train), np.vstack(Y_train.to_numpy())), axis = 1)
    Dtr = random.sample(list(Init_sampl), 2000)
    
    m = len(Dtr)
    print("n : " , m)
    # mplus =np.count_nonzero(np.array(Dtr[16]) == 'A')
    # mplus = Dtr[16].count('A')
    mplus = 0
    mneg = 0
    t=1

    index = np.random.choice(len(Dtr), 1, replace=False)  
    Dtra = np.array(Dtr)
    print(type(Dtra))
    zt = Dtra[index][0]

    print("zt : ", zt)
    
    '''
    if m%2 == 0:
        if zt[16] == 'A':
            mplus += 1;
        else:
            mneg += 1
    '''
    if zt[16]=='A':
        mplus+=1
    else:
        mneg+=1
    
    
    samp = []

    Pd = 0
    Pdd = 0
    print(zt)
    
    while(mplus + mneg < m ):
        zstar = Dtra[np.random.choice(len(Dtr), 1, replace=False)][0] #it is random sample that is obtained from St
        P = 1
        # print(zt)
        yt = zt[16]
        zt = zstar
        #print('zt:',zt)
        #print('yt:',yt)
        ##
        '''
        if P == 1:
            if zt[16] == yt:
                samp.append(zstar) #Pdash = e−y∗f0 /e−ytf0
            else:
                samp.append(zstar) #P

        if len(samp) == k:
            Pdd = q*P
            samp.append(zstar) #Pdd
        '''
        
        if P>=1 and zt[16]==yt:  # and  condition is missing..
            samp.append(zstar) #Pdash = e−y∗f0 /e−ytf0
        else:
            samp.append(zstar) #P
        
        if len(samp) == k:
            Pdd = q*P
            samp.append(zstar) #Pdd
            
        ztp1 = zstar
        if yt == 'A' and mplus<(m/2):
            mplus += 1
        else:
            if mneg<(m/2):
                mneg += 1

        if P > 1 or Pd > 1 or Pdd > 1:
            samp.append(zstar)
        
        
        ###Step 6 and step 7 are missing.................
        
        '''
        step6: i++
        train Smar by svmc and obtain learning model
        if i<k go to step 2 else output sign(fk)
        '''
        
    return samp


In [8]:
nsamp = np.array(markov_samp(X_train, y_train))
print('nsamp',nsamp)
print(nsamp.shape)
X_train = nsamp[:, 0:16]
y_train = nsamp[:, 16]
print(X_train.shape)
print(y_train.shape)

n :  2000
<class 'numpy.ndarray'>
zt :  ['-0.012309453615842696' '-0.010743015102526938' '-0.06048578502437703'
 '-0.16470367350725917' '-0.23093922668798036' '1.0377175798624314'
 '-1.0753256320031408' '-0.9735914789392905' '0.34499438865456755'
 '-0.9170705462592043' '0.20752531087631565' '-1.40778932571578'
 '-1.3059476117805662' '0.4274629460622843' '0.12008141632929248'
 '-0.4953538580534933' 'I']
['-0.012309453615842696' '-0.010743015102526938' '-0.06048578502437703'
 '-0.16470367350725917' '-0.23093922668798036' '1.0377175798624314'
 '-1.0753256320031408' '-0.9735914789392905' '0.34499438865456755'
 '-0.9170705462592043' '0.20752531087631565' '-1.40778932571578'
 '-1.3059476117805662' '0.4274629460622843' '0.12008141632929248'
 '-0.4953538580534933' 'I']
nsamp [['0.5103849674390448' '0.5944972723637817' '0.9323051058148007' ...
  '-1.0485941683840159' '-0.4953538580534933' 'J']
 ['-0.012309453615842696' '0.5944972723637817' '-0.556881230443966' ...
  '0.509639944567062' '1.97770

In [9]:
# print(y_train)
Y_train = []
for i in y_train:
    # print(i)
    Y_train.append(ord(i))
# print(Y_train)

In [10]:
print(X_test.shape)

(6000, 16)


In [11]:
# linear model

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, Y_train)

y_pred = model_linear.predict(X_test)

Y_test = []
for i in y_test:
    Y_test.append(ord(i))
# confusion matrix and accuracy

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

accuracy: 0.7941666666666667 

[[217   1   0   1   0   0   2   0   0   1   1   0   0   1   0   0   1   0
    0   1   6   1   0   0   2   0]
 [  0 183   0   3   1   1   3   6   3   1   1   0   1   0   0   2   1  12
    0   1   0   8   0   1   0   0]
 [  0   0 216   0   8   2   5   0   0   0   0   4   0   1   6   0   0   0
    0   2   4   0   0   0   0   0]
 [  2  11   0 183   0   1   0   5   0   2   0   0   1   3   2   1   0   6
    1   4   0   0   0   0   0   0]
 [  0   7   7   0 180   1  11   0   0   1   2   3   0   0   0   0   3   0
    4   1   0   0   0   2   0   3]
 [  0   2   0   2   3 208   3   2   4   2   0   0   0   0   0   3   0   0
    1   8   0   0   0   1   3   1]
 [  2   1  29   2   4   1 139   1   0   0   5   3   2   0   2   0  21   5
    2   0   1  11   0   0   0   0]
 [  0   3   6  10   0   1   5 133   0   1   5   1   3   5  11   1   1  14
    0   2   7   3   0   4   0   0]
 [  2   1   0   2   0   5   1   0 194  12   0   1   0   0   0   0   1   0
    1   0   0   0   0  

In [12]:
# non-linear model
# using rbf kernel, C=1, default value of gamma

# model
non_linear_model = SVC(kernel='rbf')

# fit
non_linear_model.fit(X_train, Y_train)

# predict
y_pred = non_linear_model.predict(X_test)
# confusion matrix and accuracy

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

accuracy: 0.8916666666666667 

[[220   0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   1
    0   1   4   0   0   2   5   0]
 [  0 195   0   1   0   2   2   1   3   0   0   0   0   0   0   1   1  15
    1   0   0   6   0   0   0   0]
 [  0   0 215   0   9   0   8   0   0   0   1   0   0   0   7   0   0   0
    0   2   6   0   0   0   0   0]
 [  1   8   0 189   0   0   3   2   1   0   0   0   0   3   4   0   0   8
    0   3   0   0   0   0   0   0]
 [  0   9   3   0 193   0   6   0   0   0   0   1   0   0   0   0   2   1
    3   0   0   0   0   3   0   4]
 [  1   1   0   0   0 217   0   1   1   1   0   0   0   0   0  11   0   0
    1   6   0   1   0   2   0   0]
 [  0   1  11   1   4   1 195   0   0   0   0   3   1   0   2   0   3   3
    0   0   1   5   0   0   0   0]
 [  0   6   0   7   0   0   5 171   0   0   2   0   1   3   3   0   4   6
    0   0   3   3   0   1   1   0]
 [  0   0   1   1   0   2   1   0 202  12   0   0   0   0   0   2   0   0
    1   0   0   0   0  

In [13]:
# non-linear model
# using rbf kernel, C=1, default value of gamma

# model
non_linear_model = SVC(kernel='poly')

# fit
non_linear_model.fit(X_train, Y_train)

# predict
y_pred = non_linear_model.predict(X_test)
# confusion matrix and accuracy

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

accuracy: 0.8413333333333334 

[[214   0   0   0   0   0   0   6   0   0   0   0   4   0   0   0   1   2
    0   0   2   2   0   3   1   0]
 [  0 185   0   0   0   3   3   9   0   1   0   0   0   0   0   1   1  13
    0   0   1   4   0   7   0   0]
 [  0   0 201   0  11   1  10   1   0   0   1   3   0   0  10   1   0   0
    1   1   5   0   0   2   0   0]
 [  2   9   0 173   0   2   2   3   0   2   0   0   0   3  11   0   0   4
    2   2   0   0   0   6   0   1]
 [  0   5   2   0 186   1   8   0   0   0   1   1   0   0   1   2   1   2
    3   0   0   0   0   8   0   4]
 [  1   1   0   3   1 201   0   0   0   5   0   0   0   1   1  17   0   0
    4   4   0   0   0   3   1   0]
 [  0   0  11   2   2   6 188   1   0   0   1   1   3   0   4   0   3   3
    0   1   0   4   0   1   0   0]
 [  0   4   1  12   0   1   3 157   0   0   2   0   0   1  17   1   1   7
    0   0   0   0   0   8   1   0]
 [  0   1   1   0   0   6   0   0 194  12   0   1   0   0   0   2   0   0
    2   0   0   0   0  